In [57]:
import numpy as np
import pandas as pd
import os
import argparse
from tqdm import tqdm
import logging

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

logging.basicConfig(level = logging.INFO)

In [73]:
class Dataset:
    def __init__(self, data_dir, dataset_name):
        
        self.path = os.path.join(data_dir, dataset_name)
        self.dataset_name = dataset_name
        
        if dataset_name is 'colon':
            self.gene_expression_values_file = "gene_values.txt"
            self.labels_file = "labels_for_each_tissue.txt"
        
        self.read_data()
        self.split_data(split_perc = 0.2)
        self.transform_data()
            
    def read_data(self):
        
        logging.info("Reading Dataset %s", self.dataset_name)
        
        if self.dataset_name is 'colon':
            with open(os.path.join(self.path, self.gene_expression_values_file), 'r') as f:
                gene_expression_values = [line.strip() for line in tqdm(f.readlines())]
                expressions = []
                for gene in gene_expression_values:
                    if gene != '':
                        expression_values = np.array(gene.split(" "))
                        expressions.append(expression_values)

            with open(os.path.join(self.path, self.labels_file), 'r') as f:
                labels = [int(line.strip()) for line in tqdm(f.readlines())]
                labels = np.array(labels)
                labels[labels>0] = 1
                labels[labels<=0] = 0
                
            self.features = np.array(expressions, dtype=np.float64).T
            self.target = labels
            
        logging.info("Reading data completed. The dataset size is %s", self.features.shape)
        
    def split_data(self, split_perc = 0.2):
        
        logging.info("Splitting the dataset into train and test sets with a split percentage of %s", split_perc)
        
        self.X_train, self.X_test, self.Y_train, self.Y_test = \
        train_test_split(self.features, self.target, test_size = split_perc, random_state = 1405)
        
        logging.info("Splitting is completed. The dimensions of the train dataset are %s", self.X_train.shape)
        
    def transform_data(self):
        
        logging.info("Standardizing the data to have to zero mean and one variance")
        
        standard_scaler = StandardScaler()
        standard_scaler.fit(self.X_train)
        self.X_train = standard_scaler.transform(self.X_train)
        self.X_test = standard_scaler.transform(self.X_test)
        
        logging.info("Standardizing is completed.")

In [74]:
colon_data = Dataset("/home/avinash/UIUC/CS466/cancer-classification/Data", 'colon')

INFO:root:Reading Dataset colon
100%|██████████| 62/62 [00:00<00:00, 274890.96it/s]
INFO:root:Reading data completed. The dataset size is (62, 2000)
INFO:root:Splitting the dataset into train and test sets with a split percentage of 0.2
INFO:root:Splitting is completed. The dimensions of the train dataset are (49, 2000)
INFO:root:Standardizing the data to have to zero mean and one variance
INFO:root:Standardizing is completed.
